# L08 09/04/24

In [1]:
import re
import pandas as pd
import numpy as np
import torchmetrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import datetime as dt
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm


/home/omirako/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-02 23:29:59.138521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 23:29:59.850249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../datasets/traffic_violations.csv', sep=',')
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,1/2/2005,1:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,1/18/2005,8:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,1/23/2005,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2/20/2005,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,3/14/2005,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52961,10/5/2011,1:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False
52962,10/5/2011,6:43,NaN,M,1986.0,25.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
52963,10/5/2011,6:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False
52964,10/5/2011,7:19,NaN,F,1986.0,25.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


# Data Preprocessing

#### N/A generation/dropping

In [3]:
df = pd.read_csv('../datasets/traffic_violations.csv', sep=',')
df = df.convert_dtypes() # force pandas to infer the data types
df = df.dropna(thresh=.8*len(df), axis = 1) # drop columns with more than 80% of na

# Insert generated data for booleans
boolean_cols = df.select_dtypes(include = bool).columns
most_frequent_values = df[boolean_cols].mode().iloc[0]
df[boolean_cols] = df[boolean_cols].fillna(most_frequent_values)

# Same for floats
float_cols = df.select_dtypes(include = float).columns
mean_values = df[float_cols].mean()
df[float_cols] = df[float_cols].fillna(mean_values)

# Same for categoricals
categorical_cols = df.select_dtypes(include='string').columns
categorical_mode_cols = df[categorical_cols].mode()
df[categorical_cols] = df[categorical_cols].fillna(categorical_mode_cols)

# if after all this, there still exists na, drop the rows
df = df.dropna(ignore_index=True)
df

,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,1/2/2005,1:55,M,1985,20,White,Speeding,Speeding,False,Citation,False,0-15 Min,False
1,1/18/2005,8:15,M,1965,40,White,Speeding,Speeding,False,Citation,False,0-15 Min,False
2,1/23/2005,23:15,M,1972,33,White,Speeding,Speeding,False,Citation,False,0-15 Min,False
3,2/20/2005,17:15,M,1986,19,White,Call for Service,Other,False,Arrest Driver,True,16-30 Min,False
4,3/14/2005,10:00,F,1984,21,White,Speeding,Speeding,False,Citation,False,0-15 Min,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49335,10/4/2011,22:22,M,1979,32,White,Speeding,Speeding,False,Arrest Driver,True,0-15 Min,False
49336,10/4/2011,22:36,F,1967,44,Hispanic,Speeding,Speeding,False,Citation,False,0-15 Min,False
49337,10/4/2011,23:25,F,1991,20,Black,Speeding,Speeding,False,Citation,False,0-15 Min,False
49338,10/5/2011,6:43,M,1986,25,White,Speeding,Speeding,False,Citation,False,0-15 Min,False


#### One hot encoding

In [4]:
# transform dates
df['stop_date'] = (pd.to_datetime(df['stop_date']) - dt.datetime(2000,1,1)).dt.total_seconds() / 3600
df['stop_time'] = pd.Series(int(a[-2:]) + int(a[:-3]) * 60 for a in df['stop_time'])

# remove all the outcomes not registered
df = df[df['stop_outcome'] != 'N/D']

# one hot encoding
categorical_features = df.select_dtypes(include='string').columns.tolist()
df = pd.get_dummies(df, prefix = categorical_features)
df

,stop_date,stop_time,driver_age_raw,driver_age,search_conducted,is_arrested,drugs_related_stop,driver_gender_F,driver_gender_M,driver_race_Asian,...,violation_Registration/plates,violation_Speeding,stop_outcome_Arrest Driver,stop_outcome_Arrest Passenger,stop_outcome_Citation,stop_outcome_No Action,stop_outcome_Warning,stop_duration_0-15 Min,stop_duration_16-30 Min,stop_duration_30+ Min
0,43872.0,115,1985,20,False,False,False,False,True,False,...,False,True,False,False,True,False,False,True,False,False
1,44256.0,495,1965,40,False,False,False,False,True,False,...,False,True,False,False,True,False,False,True,False,False
2,44376.0,1395,1972,33,False,False,False,False,True,False,...,False,True,False,False,True,False,False,True,False,False
3,45048.0,1035,1986,19,False,True,False,False,True,False,...,False,False,True,False,False,False,False,False,True,False
4,45576.0,600,1984,21,False,False,False,True,False,False,...,False,True,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49335,103056.0,1342,1979,32,False,True,False,False,True,False,...,False,True,True,False,False,False,False,True,False,False
49336,103056.0,1356,1967,44,False,False,False,True,False,False,...,False,True,False,False,True,False,False,True,False,False
49337,103056.0,1405,1991,20,False,False,False,True,False,False,...,False,True,False,False,True,False,False,True,False,False
49338,103080.0,403,1986,25,False,False,False,False,True,False,...,False,True,False,False,True,False,False,True,False,False


#### Normalization

In [5]:
# Normalize the data

X = df.drop(columns = ['stop_outcome_Arrest Passenger', 'stop_outcome_Citation', 'stop_outcome_No Action', 'stop_outcome_Warning', 'stop_outcome_Arrest Driver'])
y = df[['stop_outcome_Arrest Passenger', 'stop_outcome_Citation', 'stop_outcome_No Action', 'stop_outcome_Warning', 'stop_outcome_Arrest Driver']]

scaler = sklearn.preprocessing.StandardScaler()
numerical_cols = df.select_dtypes(include = 'number').columns
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])
# Select boolean columns
bool_cols = X.select_dtypes(include=bool).columns

# Convert boolean columns to integers
X[bool_cols] = X[bool_cols].astype(np.int64)
X

,stop_date,stop_time,driver_age_raw,driver_age,search_conducted,is_arrested,drugs_related_stop,driver_gender_F,driver_gender_M,driver_race_Asian,...,violation_raw_Violation of City/Town Ordinance,violation_raw_Warrant,violation_Equipment,violation_Moving violation,violation_Other,violation_Registration/plates,violation_Speeding,stop_duration_0-15 Min,stop_duration_16-30 Min,stop_duration_30+ Min
0,-2.004974,-1.557794,0.874360,-1.110781,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
1,-1.980003,-0.586275,-0.692779,0.454742,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
2,-1.972199,1.714690,-0.144280,-0.093191,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
3,-1.928498,0.794304,0.952717,-1.189057,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,-1.894162,-0.317829,0.796003,-1.032505,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49335,1.843818,1.579189,0.404218,-0.171467,0,1,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
49336,1.843818,1.614982,-0.536065,0.767847,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
49337,1.843818,1.740256,1.344502,-1.110781,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
49338,1.845379,-0.821485,0.952717,-0.719400,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0


In [6]:
bool_cols = y.select_dtypes(include=bool).columns

# Convert boolean columns to integers
y[bool_cols] = y[bool_cols].astype(np.int64)

y

/tmp/ipykernel_183166/3702652623.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[bool_cols] = y[bool_cols].astype(np.int64)


,stop_outcome_Arrest Passenger,stop_outcome_Citation,stop_outcome_No Action,stop_outcome_Warning,stop_outcome_Arrest Driver
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,0,0,0,1
4,0,1,0,0,0
...,...,...,...,...,...
49335,0,0,0,0,1
49336,0,1,0,0,0
49337,0,1,0,0,0
49338,0,1,0,0,0


# Definition of the network, dataloaders and validation

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the dataloader
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, target):
        self.data = torch.tensor(df.values, dtype=torch.float32) # viene messo in memoria, non si può fare con dataset molto grandi
        self.target = torch.tensor(target.values, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.argmax(self.target[idx])

# define the net
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(33, 128) # 50 input features
        self.fc2 = nn.Linear(128, 512) 
        self.fc3 = nn.Linear(512, 2048)
        self.fc4 = nn.Linear(2048, 512)
        self.fc5 = nn.Linear(512, 16)
        self.fc6 = nn.Linear(16, 5) # 1 output feature

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x
    
    import torch.nn.functional as F

# Define the validate
def validate_with_loss(model, val_loader):
    # Validate classification model:
    model.eval()
    total_loss = 0.0
    acc_micro = torchmetrics.Accuracy(task = 'multiclass', num_classes = 5, average = 'micro').to(device)
    acc_macro = torchmetrics.Accuracy(task = 'multiclass', num_classes = 5, average = 'macro').to(device)
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)  # Compute cross-entropy loss
            total_loss += loss.item() * inputs.size(0)  # Multiply by batch size to account for different batch sizes
            for i in range(len(outputs)):
                outputs[i] = torch.argmax(outputs[i])
            acc_micro.update(outputs, labels)
            acc_macro.update(outputs, labels)
            total_loss += loss.item()
    return acc_micro.compute(), acc_macro.compute(), total_loss / len(val_loader.dataset)


# Splitting of the data

In [8]:
batch_size = 512

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = Dataset(X_train, y_train.astype(int))
val_dataset = Dataset(X_val, y_val.astype(int))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Preparation of the training

In [9]:
model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
epochs = 2500
num_workers = 2
n_iter = 0


writer = SummaryWriter('Forest_experiment')


# Training

In [11]:

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    # not a multitask model, so we can use the same metric for both
    acc_micro = torchmetrics.Accuracy(task = 'multiclass', num_classes = 5, average = 'micro').to(device)
    acc_macro = torchmetrics.Accuracy(task = 'multiclass', num_classes = 5, average = 'macro').to(device)
    for i, data in enumerate(tqdm(train_loader)):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        acc_micro.update(outputs, labels)
        acc_macro.update(outputs, labels)
        
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        n_iter += 1
        writer.add_scalar('Loss/train', loss.item(), n_iter)
    acc_micro_train = acc_micro.compute()
    acc_macro_train = acc_macro.compute()
    acc_micro_val, acc_macro_val, val_loss = validate_with_loss(model, val_loader)
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/micro_train', acc_micro_train, epoch)
    writer.add_scalar('Accuracy/macro_train', acc_macro_train, epoch)
    writer.add_scalar('Accuracy/micro_val', acc_micro_val, epoch)
    writer.add_scalar('Accuracy/macro_val', acc_macro_val, epoch)
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader.dataset)}, Acc micro train: {acc_micro_train}, Acc macro train: {acc_macro_train}, Acc micro val: {acc_micro_val}, Acc macro val: {acc_macro_val}')

  0%|          | 0/76 [00:00<?, ?it/s]

100%|██████████| 76/76 [00:00<00:00, 143.50it/s]


Epoch 1, Loss: 0.0012944271155966823, Acc micro train: 0.9343647360801697, Acc macro train: 0.20000000298023224, Acc micro val: 0.003571792971342802, Acc macro val: 0.20000000298023224


100%|██████████| 76/76 [00:00<00:00, 235.49it/s]


Epoch 2, Loss: 0.0005154638845283713, Acc micro train: 0.9343904256820679, Acc macro train: 0.20000000298023224, Acc micro val: 0.003571792971342802, Acc macro val: 0.20000000298023224


...

100%|██████████| 76/76 [00:00<00:00, 245.07it/s]


Epoch 201, Loss: 0.00013580341910536054, Acc micro train: 0.9797234535217285, Acc macro train: 0.8044558167457581, Acc micro val: 0.003571792971342802, Acc macro val: 0.20000000298023224


100%|██████████| 76/76 [00:00<00:00, 238.30it/s]


Epoch 202, Loss: 0.00013396400543216772, Acc micro train: 0.9800061583518982, Acc macro train: 0.8079429268836975, Acc micro val: 0.003571792971342802, Acc macro val: 0.20000000298023224


 32%|███▏      | 24/76 [00:00<00:00, 232.10it/s]

In [ ]:
torch.argmax(torch.tensor([1.9]))

tensor(0)